# RNN(Recurrent Neural Network)
    - 참고: https://doorbw.tistory.com/158
    
    - 참고: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import sys

from tensorflow.contrib import rnn

In [5]:
tf.reset_default_graph()

# 입력값 준비 : hello
# 4개의 문자를 one-hot 인코딩으로 준비
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]


# rnn 모델 준비
cell = tf.contrib.rnn.BasicRNNCell(num_units=3)  # 4개를 입력받아서 3개를 출력



# 입력데이터준비
x_data = np.array([[h,e,l,l,o]],dtype=np.float32)
print(x_data)



# rnn 실행
outputs, _state = tf.nn.dynamic_rnn(cell,x_data,dtype=tf.float32)

sess=tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(outputs))
print("-----------------------------------")
print(sess.run(_state))

[[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]]
[[[-0.5515183  -0.5791725  -0.01904227]
  [ 0.37587062 -0.87762994  0.4774158 ]
  [ 0.70566285 -0.670852   -0.2871762 ]
  [ 0.55701065 -0.07625367  0.31357497]
  [-0.69111997 -0.22110368 -0.66647017]]]
-----------------------------------
[[-0.69111997 -0.22110368 -0.66647017]]


In [13]:
tf.reset_default_graph()

inputs=np.array([[[1,2]]])

tf.set_random_seed(777)
tf_inputs = tf.constant(inputs,dtype=tf.float32)


cell = tf.nn.rnn_cell.BasicRNNCell(num_units=3)    # 이번엔 2개 입력받아서 3개 출력
outputs,_state = tf.nn.dynamic_rnn(cell = cell, dtype = tf.float32, inputs = tf_inputs)
variables_names = [v.name for v in tf.trainable_variables()]

print(outputs)
print(_state)

print('weights')
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)
    
    
print("-----------------------------------------------------")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run,state_run = sess.run([outputs,_state])
    
    print('output values')
    print(output_run)
    
    print('\nstate values')
    print(state_run)
    
    print('\nweights')
    values = sess.run(variables_names)
    
    for k,v in zip(variables_names,values):
        print(k,v)

Tensor("rnn/transpose_1:0", shape=(1, 1, 3), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(1, 3), dtype=float32)
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(5, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>
-----------------------------------------------------
output values
[[[-0.9314169   0.75578666 -0.6819246 ]]]

state values
[[-0.9314169   0.75578666 -0.6819246 ]]

weights
rnn/basic_rnn_cell/kernel:0 [[-0.62831575  0.38538355  0.79733914]
 [-0.5203329   0.30046564 -0.8150209 ]
 [ 0.39399797  0.16670114  0.4062907 ]
 [-0.6391754   0.8460203   0.5266966 ]
 [ 0.41124135  0.66347724 -0.0210759 ]]
rnn/basic_rnn_cell/bias:0 [0. 0. 0.]


In [14]:
tf.reset_default_graph()

tf.set_random_seed(777)


# I work at google  = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]   # 4개의 단어를 4개의 []로 one-hot 인코딩
# I google at work  = [[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]]


inputs = np.array([[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],
                  [[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]]])


tf_inputs = tf.constant(inputs,dtype=tf.float32)
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=3)

outputs,_state = tf.nn.dynamic_rnn(cell = cell, dtype = tf.float32, inputs = tf_inputs)
variables_names = [v.name for v in tf.trainable_variables()]


print(outputs)
print(_state)


print('weights')
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)
    

Tensor("rnn/transpose_1:0", shape=(2, 4, 3), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(2, 3), dtype=float32)
weights
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


In [32]:
# hi hello 학습

tf.reset_default_graph()
tf.set_random_seed(777)



# 파라미터
num_classes = 5
input_dim = 5
hidden_size = 5
batch_size = 1
sequence_len = 7
lr = 0.1



idx2char = ['h','i','e','l','o']
x_one_hot = [[
    [1,0,0,0,0],    # h
    [0,1,0,0,0],    # i
    [1,0,0,0,0],    # h
    [0,0,1,0,0],    # e
    [0,0,0,1,0],    # l
    [0,0,0,1,0],    # l   
    [0,0,0,0,1]     # o
]]


x_data = [[0,1,0,2,3,3]]  # hihell   
y_data = [[0,1,0,2,3,3,4]]  # hihello



# X,y 변수
X = tf.placeholder(tf.float32,shape=[None,sequence_len, input_dim])
y = tf.placeholder(tf.int32,[None,sequence_len])


cell = tf.contrib.rnn.BasicRNNCell(num_units= hidden_size)
outputs, _state = tf.nn.dynamic_rnn(cell= cell,inputs=X,dtype=tf.float32,initial_state= cell.zero_state(1,tf.float32))


print(outputs)




# 평면화
X_for_fc = tf.reshape(outputs,[-1,hidden_size])


# FC Layer
# W, b, logit
outputs = tf.contrib.layers.fully_connected(inputs=X_for_fc, num_outputs=5 , activation_fn=None)

outputs = tf.reshape(outputs,[batch_size,sequence_len,num_classes])
weights = tf.ones([batch_size,sequence_len])
seq_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y , weights=weights )

loss = tf.reduce_mean(seq_loss)
train = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

Tensor("rnn/transpose_1:0", shape=(1, 7, 5), dtype=float32)


In [34]:
prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        
        
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

0 loss: 1.6383449 prediction:  [[0 3 3 3 3 3 2]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hllllle
1 loss: 1.386576 prediction:  [[0 0 0 3 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hhhlllo
2 loss: 1.1635326 prediction:  [[0 0 0 3 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hhhlllo
3 loss: 0.9318157 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
4 loss: 0.70485556 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
5 loss: 0.50350326 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
6 loss: 0.35185286 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
7 loss: 0.25121334 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
8 loss: 0.18412563 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
9 loss: 0.13794504 predi

120 loss: 0.0009382633 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
121 loss: 0.0009323109 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
122 loss: 0.00092639215 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
123 loss: 0.0009205927 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
124 loss: 0.00091479305 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
125 loss: 0.0009091124 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
126 loss: 0.00090344885 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
127 loss: 0.0008978191 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
128 loss: 0.0008922573 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Predi

244 loss: 0.00048640868 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
245 loss: 0.0004842473 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
246 loss: 0.00048205184 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
247 loss: 0.0004799585 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
248 loss: 0.00047781406 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
249 loss: 0.0004757037 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
250 loss: 0.00047362735 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
251 loss: 0.00047149992 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
252 loss: 0.0004694746 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Pre

370 loss: 0.00029704758 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
371 loss: 0.00029606023 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
372 loss: 0.0002950218 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
373 loss: 0.00029403443 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
374 loss: 0.00029303008 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
375 loss: 0.00029202565 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
376 loss: 0.00029103828 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
377 loss: 0.00029005093 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
378 loss: 0.0002890806 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	P

492 loss: 0.00020363442 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
493 loss: 0.00020307259 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
494 loss: 0.00020251072 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
495 loss: 0.00020191485 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
496 loss: 0.00020138707 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
497 loss: 0.00020079115 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
498 loss: 0.00020026336 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
499 loss: 0.00019970152 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
500 loss: 0.00019917371 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]


...

1587 loss: 3.2185864e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1588 loss: 3.216884e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1589 loss: 3.2117747e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1590 loss: 3.204963e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1591 loss: 3.1998545e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1592 loss: 3.1998545e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1593 loss: 3.198151e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1594 loss: 3.1947453e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1595 loss: 3.189637e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3,

1688 loss: 2.8814073e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1689 loss: 2.8762983e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1690 loss: 2.8728928e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1691 loss: 2.8728928e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1692 loss: 2.8677838e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1693 loss: 2.866081e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1694 loss: 2.8609722e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1695 loss: 2.8558637e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1696 loss: 2.8541606e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3,

1784 loss: 2.6021266e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1785 loss: 2.5987205e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1786 loss: 2.5970176e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1787 loss: 2.593612e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1788 loss: 2.591909e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1789 loss: 2.588503e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1790 loss: 2.5868e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1791 loss: 2.5833942e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1792 loss: 2.5816913e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4

1882 loss: 2.3534974e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1883 loss: 2.3534974e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1884 loss: 2.3517947e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1885 loss: 2.3517947e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1886 loss: 2.34328e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1887 loss: 2.339874e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1888 loss: 2.339874e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1889 loss: 2.338171e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1890 loss: 2.3330624e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 

	Prediction str:  hihello
1980 loss: 2.1389269e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1981 loss: 2.135521e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1982 loss: 2.135521e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1983 loss: 2.1321152e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1984 loss: 2.1304122e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1985 loss: 2.1287093e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1986 loss: 2.1270063e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1987 loss: 2.1270063e-05 prediction:  [[0 1 0 2 3 3 4]] true Y:  [[0, 1, 0, 2, 3, 3, 4]]
	Prediction str:  hihello
1988 loss: 2.1270063e-05 prediction:  [[0 1 0 2 3 3 4]] 

In [38]:
# LSTM

tf.reset_default_graph()

tf.set_random_seed(777)

inputs = np.array([[[1,0]]])

tf_inputs = tf.constant(inputs,dtype=tf.float32)
cell = tf.nn.rnn_cell.LSTMCell(num_units=1)
outputs,_state = tf.nn.dynamic_rnn(cell=cell,dtype=tf.float32,inputs=tf_inputs)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outputs_run, state_run = sess.run([outputs,_state])
    
    print('output values')
    print(output_run)
    print('\nhidden values')
    print(state_run.h)
    print('\nmemory cell')
    print(state_run.c)


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
output values
[[[-0.9314169   0.75578666 -0.6819246 ]]]

hidden values
[[0.09927537]]

memory cell
[[0.18134572]]


### 사례 : LSTM을 이용한 문장분류

In [39]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the thiadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]

In [40]:
df = pd.DataFrame(paragraph_dict_list)

df = df[['paragraph','category']]
df

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food
5,the garlic fries were a great starter (and a h...,food
6,our meal was excellent i had the pasta ai form...,food
7,what i enjoy most about palo alto is so many r...,food
8,the drinks came out fairly quickly a good two ...,food
9,despite the not so good burger the service was...,food


In [42]:
results = set()
df['paragraph'].str.lower().str.split().apply(results.update)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: paragraph, dtype: object

In [43]:
results

{'(and',
 '1',
 '1.5',
 '1.5x',
 '10-9',
 '14',
 '2',
 '2014',
 '33/1',
 'a',
 'about',
 'action',
 'after',
 'against',
 'ago',
 'ai',
 'alisson',
 'all',
 'almost',
 'also',
 'although',
 'alto',
 'amazing',
 'american',
 'and',
 'angelique',
 'any',
 'apps',
 'are',
 'area',
 'around',
 'as',
 'at',
 'australia',
 'avoided',
 'azarenka',
 'back',
 'battle',
 'be',
 'beef',
 'been',
 'before',
 'began',
 'began—too',
 'being',
 'better',
 'between',
 'big',
 'biscuits',
 'bit',
 'bookmark',
 'bookmarked',
 'both',
 'bottom',
 'brazilian',
 'bread',
 'breakthroughs',
 'briton',
 'brunch',
 'burger',
 'burgers',
 'business',
 'busy',
 'but',
 'by',
 'came',
 'campaigns',
 'can',
 'card',
 'caroline',
 'case',
 'certainly',
 'champion',
 'champions',
 'chance',
 'chances',
 'chicken',
 'city',
 'clay',
 'clean',
 'commented',
 'competition',
 'complain',
 'confidence',
 'contrast',
 'could',
 'course',
 'courteous',
 'courts',
 'coworkers',
 'crisp',
 'croatia',
 'crumbs',
 'culminating

In [45]:
idx2word = dict(enumerate(results))
idx2word

{0: 'would',
 1: 'modric',
 2: 'long',
 3: 'clean',
 4: 'wanted',
 5: 'knock',
 6: 'jose',
 7: 'played',
 8: 'smash',
 9: 'up',
 10: 'truly',
 11: 'any',
 12: 'with',
 13: 'three',
 14: 'drinks',
 15: 'quarter-final',
 16: 'this',
 17: 'could',
 18: 'in',
 19: 'player',
 20: 'hold',
 21: 'around',
 22: 'delivered',
 23: 'chances',
 24: 'that',
 25: 'our',
 26: 'north',
 27: 'team',
 28: 'said',
 29: 'odds',
 30: 'outside',
 31: 'apps',
 32: 'clay',
 33: 'from',
 34: 'turnaround',
 35: 'course',
 36: 'opened',
 37: 'despite',
 38: 'sliders',
 39: '14',
 40: 'soft',
 41: 'within',
 42: 'russia',
 43: 'first',
 44: 'many',
 45: 'garlic',
 46: 'seeding',
 47: 'however',
 48: 'place',
 49: 'coworkers',
 50: 'struggled',
 51: 'chance',
 52: 'look',
 53: 'hip',
 54: 'swing',
 55: 'year—and',
 56: 'no83',
 57: 'flavors',
 58: 'pleasantly',
 59: 'san',
 60: 'to',
 61: 'downtown',
 62: 'week',
 63: 'prepare',
 64: 'went',
 65: 'minutes',
 66: 'dare',
 67: 'taste',
 68: 'left',
 69: '33/1',
 70: 

In [46]:
idx2word[396]
idx2word[100]

'time'

In [47]:
word2idx = {v:k for k,v in idx2word.items()}
word2idx

{'would': 0,
 'modric': 1,
 'long': 2,
 'clean': 3,
 'wanted': 4,
 'knock': 5,
 'jose': 6,
 'played': 7,
 'smash': 8,
 'up': 9,
 'truly': 10,
 'any': 11,
 'with': 12,
 'three': 13,
 'drinks': 14,
 'quarter-final': 15,
 'this': 16,
 'could': 17,
 'in': 18,
 'player': 19,
 'hold': 20,
 'around': 21,
 'delivered': 22,
 'chances': 23,
 'that': 24,
 'our': 25,
 'north': 26,
 'team': 27,
 'said': 28,
 'odds': 29,
 'outside': 30,
 'apps': 31,
 'clay': 32,
 'from': 33,
 'turnaround': 34,
 'course': 35,
 'opened': 36,
 'despite': 37,
 'sliders': 38,
 '14': 39,
 'soft': 40,
 'within': 41,
 'russia': 42,
 'first': 43,
 'many': 44,
 'garlic': 45,
 'seeding': 46,
 'however': 47,
 'place': 48,
 'coworkers': 49,
 'struggled': 50,
 'chance': 51,
 'look': 52,
 'hip': 53,
 'swing': 54,
 'year—and': 55,
 'no83': 56,
 'flavors': 57,
 'pleasantly': 58,
 'san': 59,
 'to': 60,
 'downtown': 61,
 'week': 62,
 'prepare': 63,
 'went': 64,
 'minutes': 65,
 'dare': 66,
 'taste': 67,
 'left': 68,
 '33/1': 69,
 'sis

In [61]:
def encode_paragraph(paragraph):
    words = paragraph.split(' ')
    encoded = []
    
    for word in words:
        encoded.append([word2idx[word]])
        
    return encoded    



def encode_category(category):
    if category == 'food':
        return [1,0]
    else :
        return [0,1]
    
    
 
def word_cnt(paragraph):
    return len(paragraph.split(' '))

In [62]:
df['enc_paragraph'] = df.paragraph.apply(encode_paragraph)
df['enc_category'] = df.category.apply(encode_category)
df['seq_length'] = df.paragraph.apply(word_cnt)
df

,paragraph,category,enc_paragraph,enc_category,seq_length
0,dishplace is located in sunnyvale downtown the...,food,"[[471], [98], [268], [18], [333], [61], [435],...","[1, 0]",53
1,service can be slower during busy hours but ou...,food,"[[375], [372], [515], [194], [391], [217], [22...","[1, 0]",19
2,portions are huge both french toast and their ...,food,"[[90], [460], [252], [431], [400], [357], [254...","[1, 0]",42
3,we started with apps going the chicken and waf...,food,"[[483], [358], [12], [31], [182], [457], [192]...","[1, 0]",43
4,the biscuits and gravy was too salty two peopl...,food,"[[457], [525], [254], [350], [92], [139], [291...","[1, 0]",82
5,the garlic fries were a great starter (and a h...,food,"[[457], [45], [112], [181], [277], [256], [473...","[1, 0]",24
6,our meal was excellent i had the pasta ai form...,food,"[[25], [319], [92], [242], [188], [86], [457],...","[1, 0]",50
7,what i enjoy most about palo alto is so many r...,food,"[[510], [188], [299], [127], [484], [75], [495...","[1, 0]",43
8,the drinks came out fairly quickly a good two ...,food,"[[457], [14], [234], [422], [424], [278], [277...","[1, 0]",49
9,despite the not so good burger the service was...,food,"[[37], [457], [376], [371], [245], [491], [457...","[1, 0]",82


In [63]:
# 문장의 최대 길이

max_word_cnt = 0

for row in df['paragraph']:
    if len(row.split(' ')) > max_word_cnt:
        max_word_cnt = len(row.split(' '))
        
print(max_word_cnt)        

91


In [64]:
# 문장의 길이가 91개가 안될 경우 -1로 패딩처리

def sequence_padding(enc_paragraph):
    seq_length = len(enc_paragraph)
    for i in range(seq_length, max_word_cnt):
        enc_paragraph.append([-1])
        
    return enc_paragraph    
        

In [65]:
df['enc_paragraph'] = df.enc_paragraph.apply(sequence_padding)

In [66]:
print(df['enc_paragraph'][0])

[[471], [98], [268], [18], [333], [61], [435], [98], [488], [21], [457], [311], [208], [159], [372], [515], [249], [60], [184], [391], [326], [185], [228], [447], [70], [254], [188], [234], [60], [16], [48], [500], [303], [347], [277], [126], [421], [181], [472], [217], [371], [188], [505], [231], [336], [276], [464], [427], [339], [457], [244], [60], [408], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]


In [67]:
# 입력값들을 배열로 변환

enc_paragraph = np.array(df.enc_paragraph.tolist())
enc_category = np.array(df.enc_category.tolist())

seq_length = np.array(df.seq_length.tolist())



In [68]:
train_X = enc_paragraph

train_y = enc_category

print(train_X.shape)
print(train_y.shape)

(20, 91, 1)
(20, 2)


In [75]:
# 모델 구축

tf.reset_default_graph()
tf.set_random_seed(777)

lr = 0.001
n_epochs = 300


X = tf.placeholder(tf.float32,[None,max_word_cnt,1])
y = tf.placeholder(tf.int32,[None,2])

cell = tf.nn.rnn_cell.LSTMCell(num_units=64)
outputs, _state = tf.nn.dynamic_rnn(cell,X,dtype=tf.float32,sequence_length=seq_length)


# 간단하게 한줄로 마술과도 같은 그런 모듈 layer...
# dense() -> 하나의 히든계층을 생성.
dense_layer = tf.layers.dense(_state.h, 32)
logits = tf.layers.dense(dense_layer,2)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = logits)
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

In [76]:
print(X)
print(_state)
print(dense_layer)
print(logits)

Tensor("Placeholder:0", shape=(?, 91, 1), dtype=float32)
LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 64) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 64) dtype=float32>)
Tensor("dense/BiasAdd:0", shape=(?, 32), dtype=float32)
Tensor("dense_1/BiasAdd:0", shape=(?, 2), dtype=float32)


In [77]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        _, cost = sess.run([train, loss], feed_dict={X:train_X, y:train_y})
        if epoch % 50 == 0:
            pred = tf.nn.softmax(logits)
            correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_pred, "float"))
            cur_acc = accuracy.eval({X:train_X, y:train_y})
            print("epoch:" + str(epoch) + ", loss:" + str(cost) + ", accuracy:" + str(cur_acc))

epoch:0, loss:0.7184518, accuracy:0.5
epoch:50, loss:0.42547274, accuracy:0.75
epoch:100, loss:0.026048, accuracy:1.0
epoch:150, loss:0.0010713647, accuracy:1.0
epoch:200, loss:0.00024955155, accuracy:1.0
epoch:250, loss:0.00010911899, accuracy:1.0
